In [133]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Bidirectional

In [134]:
def split_sequence(sequence, n_steps):
  X, y = list(), list()
  for i in range(len(sequence)):
    # find the end of this pattern
    end_ix = i + n_steps
    # check if we are beyond the sequence
    if end_ix > len(sequence)-1:
      break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
    X.append(seq_x)
    y.append(seq_y)
  return np.array(X), np.array(y)

In [135]:
import numpy as np
import pandas as pd


def evaluate():
    # Input the csv file
    """
    Sample evaluation function
    Don't modify this function
    """
    df = pd.read_csv('/content/sample_input.csv')

    actual_close = np.loadtxt('/content/sample_close.txt')

    pred_close = predict_func(df)

    # Calculation of squared_error
    actual_close = np.array(actual_close)
    pred_close = np.array(pred_close)
    mean_square_error = np.mean(np.square(actual_close-pred_close))


    pred_prev = [df['Close'].iloc[-1]]
    pred_prev.append(pred_close[0])
    pred_curr = pred_close

    actual_prev = [df['Close'].iloc[-1]]
    actual_prev.append(actual_close[0])
    actual_curr = actual_close

    # Calculation of directional_accuracy
    pred_dir = np.array(pred_curr)-np.array(pred_prev)
    actual_dir = np.array(actual_curr)-np.array(actual_prev)
    dir_accuracy = np.mean((pred_dir*actual_dir)>0)*100

    print(f'Mean Square Error: {mean_square_error:.6f}\nDirectional Accuracy: {dir_accuracy:.1f}')


def predict_func(data):
    """
    Modify this function to predict closing prices for next 2 samples.
    Take care of null values in the sample_input.csv file which are listed as NAN in the dataframe passed to you
    Args:
        data (pandas Dataframe): contains the 50 continuous time series values for a stock index

    Returns:
        list (2 values): your prediction for closing price of next 2 samples
    """
    data = data.interpolate(order=3) #Interpolating the dataframe to fill in the Null Values
    raw_seq = np.array(df['Close']) #Selecting the column in focus
    # choose a number of time steps
    n_steps = 3
    # split into samples
    X, y = split_sequence(raw_seq, n_steps)
    # reshape the data
    n_features = 1
    X = X.reshape((X.shape[0], X.shape[1], n_features))
    '''Bidirectional LSTM is used to fit in the data with 50 subunits
    and activation function as ReLU(Rectified Linear Unit).'''
    model = Sequential()
    model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(n_steps, n_features)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    # fit model
    model.fit(X, y, epochs=200, verbose=0)
    # demonstrate prediction
    avd = np.array(df['Close']) # Avd will contain the 2 predicted value with the original data
    for i in range(2):
      x_input = avd[47+i:50+i]
      x_input = x_input.reshape((1, n_steps, n_features))
      yhat = model.predict(x_input, verbose=0)
      avd = np.append(avd,yhat) # Adding the new predicted value to the array
      print(avd[len(avd)-1])
    print('R2_score = ',r2_score([7232.200195,7277.899902],[avd[50],avd[51]]))
    return [avd[50],avd[51]]


if __name__== "__main__":
    evaluate()

7232.47705078125
7269.89794921875
R2_score =  0.9386075877932172
Mean Square Error: 32.053949
Directional Accuracy: 100.0
